# Trabajo Final - "Titanic"

### Plan de trabajo


1. Apertura archivos y analisis (tamaños, estructura matriz, etc)
    - Analisis estructura de la matriz y cantidad de datos en cada archivo
2. Análisis de datos dentro del train 
    - Cantidad de datos nulos y análisis de datos para determinar conversiones, agregado/borrado/modificación de columnas, identificación de datos "Outliers" (o atípicos) o de necesidad de normalizar por tener magnitudes muy diferentes
    - Definición de acciones a implementar para preparar los datos previo al entrenamiento
3. Borrado de columnas que no utilizaremos

----------1er etapa MATI

4. Completar columnas con datos faltantes
5. Transformar columnas con datos objeto a varias columnas con datos numéricos (ceros y unos)
6. Ejecución y búsqueda del mejor modelo de aprendizaje
7. Ejecución del mejor modelo encontrado
8. Guardado del modelo
9. Apertura  del modelo guardado y prueba con 2 datos nuevos

### 1. Apertura de archivos y análisis

In [107]:
import pandas as pd
import numpy as np

In [108]:
pd.set_option('display.max_rows', None)

In [109]:
train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")
y_test = pd.read_csv("data/gender_submission.csv")

#### Analisis estructura de la matriz y datos de los archivos

In [110]:
train.shape

(891, 12)

In [111]:
X_test.shape

(418, 11)

In [112]:
y_test.shape

(418, 2)

>___Conclusiones: Entre train y test hay 1309 pasajeros, train con 891 y test con 418 (60%/40%), con 12 columnas en el train___

>___Dado que el train da 60% y el test 40%, ¿deberiamos unir todo y dividir luego en 80/20 para tener mas datos para entrenar?___

In [113]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


***Columnas y contenidos***

Variable	   Definición		        Llave
- survival	     Supervivencia		0 = No, 1 = Sí
- pclass		Clase de billete	1 = 1º, 2 = 2º, 3 = 3º
- sex		    Sexo	
- age		    Edad en años	
- sibsp		    # de hermanos / cónyuges a bordo del Titanic	
- parch		    # de padres / hijos a bordo del Titanic	
- ticket		Número de billete	
- fare		    Tarifa de pasajero	
- cabin		    Número de cabina	
- embarked	    Puerto de embarque	C = Cherburgo, Q = Queenstown, S = Southampton

In [8]:
train.sort_values(by=["Pclass","Fare"], ascending=[True, True]).groupby("Pclass")["Fare"].mean()

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64

In [114]:
cortes=pd.cut(train.Pclass,3)

In [115]:
x=train.groupby(["Pclass","Fare"],squeeze=True)["Fare"].value_counts()
#print(x)

<ipython-input-115-d8a004d45806>:1: FutureWarning: The `squeeze` parameter is deprecated and will be removed in a future version.
  x=train.groupby(["Pclass","Fare"],squeeze=True)["Fare"].value_counts()


In [11]:
#train.sort_values(by=["Pclass","Fare"], ascending=[True,True])

In [12]:
#train.Pclass

In [116]:
train.groupby(("Embarked")).count()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin
Embarked,,,,,,,,,,,
C,168,168,168,168,168,130,168,168,168,168,69
Q,77,77,77,77,77,28,77,77,77,77,4
S,644,644,644,644,644,554,644,644,644,644,129


In [117]:
train.groupby(("Embarked")).mean()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Embarked,,,,,,,
C,445.357143,0.553571,1.886905,30.814769,0.386905,0.363095,59.954144
Q,417.896104,0.389610,2.909091,28.089286,0.428571,0.168831,13.276030
S,449.527950,0.336957,2.350932,29.445397,0.571429,0.413043,27.079812


##### El archivo Test tiene 418 filas por 11 columnas

In [118]:
X_test.shape

(418, 11)

In [119]:
X_test.head(4)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S


In [120]:
y_test.shape

(418, 2)

In [121]:
y_test.head(4)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [20]:
train.Pclass.unique()

array([3, 1, 2], dtype=int64)

In [21]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [22]:
train.Parch.value_counts(dropna=True) 

0    678
1    118
2     80
3      5
5      5
4      4
6      1
Name: Parch, dtype: int64

In [23]:
train.groupby(by=["Parch"]).count()  #(by=["b"]).sum()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Ticket,Fare,Cabin,Embarked
Parch,,,,,,,,,,,
0,678,678,678,678,678,521,678,678,678,141,676
1,118,118,118,118,118,110,118,118,118,39,118
2,80,80,80,80,80,68,80,80,80,23,80
3,5,5,5,5,5,5,5,5,5,0,5
4,4,4,4,4,4,4,4,4,4,1,4
5,5,5,5,5,5,5,5,5,5,0,5
6,1,1,1,1,1,1,1,1,1,0,1


In [24]:
np.count_nonzero(train.Parch)

213

In [25]:
np.count_nonzero(train.SibSp) 

283

In [123]:
train["Flia"]=train["SibSp"]+train["Parch"]
X_test["Flia"]=X_test["SibSp"]+X_test["Parch"]

In [124]:
X_test.head(1)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
 11  Flia         418 non-null    int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 39.3+ KB


In [28]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Flia
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1


In [29]:
np.sum(train.Flia)

806

In [30]:
np.sum(train.Parch)

340

In [31]:
np.sum(train.SibSp)

466

In [125]:
train.groupby(["Flia"]).sum()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
Flia,,,,,,,
0,245946,163,1289,13017.00,0,0,11407.3238
1,66467,89,309,4363.42,123,38,8032.9547
2,45732,59,227,2421.33,89,115,4048.6332
3,14916,21,60,493.42,39,48,1591.0708
4,5518,3,40,229.00,32,28,871.4168
5,8803,3,57,405.00,62,48,1621.9000
6,4269,4,36,182.00,39,33,352.4000
7,2363,0,18,94.00,26,16,281.4000
10,3372,0,21,0.00,56,14,486.8500


##### Conclusiones
- Sacar columna Name, Ticket,Fare, Cabin, Sibsp y Parch (porque estos últimos dos estan ya sumados en Flia

In [126]:
# Borrado de columnas que no utilizaremos. 
train = train.drop(['Name', 'Ticket', 'Fare', 'Cabin','SibSp','Parch'], axis=1) 
train.head(20)

,PassengerId,Survived,Pclass,Sex,Age,Embarked,Flia
0,1,0,3,male,22.0,S,1
1,2,1,1,female,38.0,C,1
2,3,1,3,female,26.0,S,0
3,4,1,1,female,35.0,S,1
4,5,0,3,male,35.0,S,0
5,6,0,3,male,NaN,Q,0
6,7,0,1,male,54.0,S,0
7,8,0,3,male,2.0,S,4
8,9,1,3,female,27.0,S,2
9,10,1,2,female,14.0,C,1


Split del train en X_train e y_train

In [127]:
y_train=train[["Survived"]]
y_train.head(4)

,Survived
0,0
1,1
2,1
3,1


In [128]:
X_train=train.drop(["Survived"],axis=1)
X_train.head(4)

,PassengerId,Pclass,Sex,Age,Embarked,Flia
0,1,3,male,22.0,S,1
1,2,1,female,38.0,C,1
2,3,3,female,26.0,S,0
3,4,1,female,35.0,S,1


- Sex y Embarked: transformar a 2/3 columnas numéricas con unos y ceros
- Age: Agregar 177 datos faltantes en train y los faltantes en test   

In [129]:
X_train.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [37]:
X_train.shape

(891, 6)

In [38]:
y_train.shape

(891, 1)

In [130]:
X_train.dropna(subset=["Embarked"],inplace=True) #borro 2 filas con Embarked nulo.

In [131]:
borradas = y_train.sample(2)
borradas

,Survived
766,0
812,0


In [132]:
y_train.drop(borradas.index, inplace=True) #para equiparar borro 2 filas aleatorias.

C:\Users\CTI0212\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [42]:
X_train.shape

(889, 6)

In [43]:
y_train.shape

(889, 1)

In [44]:
X_test.Embarked.unique()

array(['Q', 'S', 'C'], dtype=object)

In [45]:
X_test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Flia
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,2


In [133]:
# CORRECCIÓN AL X_test.
# Borrado de columnas que no utilizaremos. 
X_test = X_test.drop(['Name', 'Ticket', 'Fare', 'Cabin','SibSp','Parch'], axis=1) 
X_test.head(1)

,PassengerId,Pclass,Sex,Age,Embarked,Flia
0,892,3,male,34.5,Q,0


In [134]:
X_train.Embarked.unique()

array(['S', 'C', 'Q'], dtype=object)

In [135]:
# Con onehotencoder
# Transformar columnas con datos objeto a varias columnas con datos numéricos (ceros y unos).
# Esas serían las columnas: sex, embarked.
# Recordar ahcerlo primero en train y luego en test
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder
# Definimos el transformador para OneHotEncoder de foma muy similar a como trabajamos SimpleImputer o kNNImputer
tdummy_2 = ("onehot",OneHotEncoder(sparse = False), ['Sex', 'Embarked'])

# Creamos el transformador de columnas
transformador_columnas_dummy2 = ColumnTransformer(transformers = [tdummy_2], remainder = 'passthrough')
# Lo entrenamos con fit en X_train
transformador_columnas_dummy2.fit(X_train)
# Lo aplicamos al X_train
X_train = transformador_columnas_dummy2.transform(X_train)
#dummy_2[[0:1, ]]
X_train[0:20,]


array([[ 0.,  1.,  0.,  0.,  1.,  1.,  3., 22.,  1.],
       [ 1.,  0.,  1.,  0.,  0.,  2.,  1., 38.,  1.],
       [ 1.,  0.,  0.,  0.,  1.,  3.,  3., 26.,  0.],
       [ 1.,  0.,  0.,  0.,  1.,  4.,  1., 35.,  1.],
       [ 0.,  1.,  0.,  0.,  1.,  5.,  3., 35.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  6.,  3., nan,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  7.,  1., 54.,  0.],
       [ 0.,  1.,  0.,  0.,  1.,  8.,  3.,  2.,  4.],
       [ 1.,  0.,  0.,  0.,  1.,  9.,  3., 27.,  2.],
       [ 1.,  0.,  1.,  0.,  0., 10.,  2., 14.,  1.],
       [ 1.,  0.,  0.,  0.,  1., 11.,  3.,  4.,  2.],
       [ 1.,  0.,  0.,  0.,  1., 12.,  1., 58.,  0.],
       [ 0.,  1.,  0.,  0.,  1., 13.,  3., 20.,  0.],
       [ 0.,  1.,  0.,  0.,  1., 14.,  3., 39.,  6.],
       [ 1.,  0.,  0.,  0.,  1., 15.,  3., 14.,  0.],
       [ 1.,  0.,  0.,  0.,  1., 16.,  2., 55.,  0.],
       [ 0.,  1.,  0.,  1.,  0., 17.,  3.,  2.,  5.],
       [ 0.,  1.,  0.,  0.,  1., 18.,  2., nan,  0.],
       [ 1.,  0.,  0.,  0., 

In [136]:
# Sexf,Sexm,Ec,Eq,Es,Passid,Pclass,Age,Flia
X_train=pd.DataFrame(X_train)
X_train.columns = ["Sexf","Sexm","Ec","Eq","Es","Pid","Pclass","Age","Flia"]
X_train

,Sexf,Sexm,Ec,Eq,Es,Pid,Pclass,Age,Flia
0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,22.00,1.0
1,1.0,0.0,1.0,0.0,0.0,2.0,1.0,38.00,1.0
2,1.0,0.0,0.0,0.0,1.0,3.0,3.0,26.00,0.0
3,1.0,0.0,0.0,0.0,1.0,4.0,1.0,35.00,1.0
4,0.0,1.0,0.0,0.0,1.0,5.0,3.0,35.00,0.0
5,0.0,1.0,0.0,1.0,0.0,6.0,3.0,NaN,0.0
6,0.0,1.0,0.0,0.0,1.0,7.0,1.0,54.00,0.0
7,0.0,1.0,0.0,0.0,1.0,8.0,3.0,2.00,4.0
8,1.0,0.0,0.0,0.0,1.0,9.0,3.0,27.00,2.0
9,1.0,0.0,1.0,0.0,0.0,10.0,2.0,14.00,1.0


In [137]:
# Lo aplicamos al y_test
X_test = transformador_columnas_dummy2.transform(X_test)
X_test

array([[ 0. ,  1. ,  0. , ...,  3. , 34.5,  0. ],
       [ 1. ,  0. ,  0. , ...,  3. , 47. ,  1. ],
       [ 0. ,  1. ,  0. , ...,  2. , 62. ,  0. ],
       ...,
       [ 0. ,  1. ,  0. , ...,  3. , 38.5,  0. ],
       [ 0. ,  1. ,  0. , ...,  3. ,  nan,  0. ],
       [ 0. ,  1. ,  1. , ...,  3. ,  nan,  2. ]])

In [51]:
X_test=pd.DataFrame(X_test)
X_test.columns = ["Sexf","Sexm","Ec","Eq","Es","Pid","Pclass","Age","Flia"]
X_test

,Sexf,Sexm,Ec,Eq,Es,Pid,Pclass,Age,Flia
0,0.0,1.0,0.0,1.0,0.0,892.0,3.0,34.50,0.0
1,1.0,0.0,0.0,0.0,1.0,893.0,3.0,47.00,1.0
2,0.0,1.0,0.0,1.0,0.0,894.0,2.0,62.00,0.0
3,0.0,1.0,0.0,0.0,1.0,895.0,3.0,27.00,0.0
4,1.0,0.0,0.0,0.0,1.0,896.0,3.0,22.00,2.0
5,0.0,1.0,0.0,0.0,1.0,897.0,3.0,14.00,0.0
6,1.0,0.0,0.0,1.0,0.0,898.0,3.0,30.00,0.0
7,0.0,1.0,0.0,0.0,1.0,899.0,2.0,26.00,2.0
8,1.0,0.0,1.0,0.0,0.0,900.0,3.0,18.00,0.0
9,0.0,1.0,0.0,0.0,1.0,901.0,3.0,21.00,2.0


In [138]:
# Completar columna Age con modelo KnnImputer con datos faltantes. 
# Recordar ahcerlo primero en train y luego en test.
# Importar biblioteca KnnImputer.
from sklearn.impute import KNNImputer
# Asignar a una variable el imputador.
imputador_knn = KNNImputer(missing_values = np.NaN, n_neighbors = 2)
# Hacer el fit sobre el X_train.
imputador_knn.fit(X_train)
# Hacer el transform.
Train_Transformado = imputador_knn.transform(X_train)
# Train_Transformado 
X_train = pd.DataFrame(Train_Transformado, columns = X_train.columns)
X_train

,Sexf,Sexm,Ec,Eq,Es,Pid,Pclass,Age,Flia
0,0.0,1.0,0.0,0.0,1.0,1.0,3.0,22.00,1.0
1,1.0,0.0,1.0,0.0,0.0,2.0,1.0,38.00,1.0
2,1.0,0.0,0.0,0.0,1.0,3.0,3.0,26.00,0.0
3,1.0,0.0,0.0,0.0,1.0,4.0,1.0,35.00,1.0
4,0.0,1.0,0.0,0.0,1.0,5.0,3.0,35.00,0.0
5,0.0,1.0,0.0,1.0,0.0,6.0,3.0,44.50,0.0
6,0.0,1.0,0.0,0.0,1.0,7.0,1.0,54.00,0.0
7,0.0,1.0,0.0,0.0,1.0,8.0,3.0,2.00,4.0
8,1.0,0.0,0.0,0.0,1.0,9.0,3.0,27.00,2.0
9,1.0,0.0,1.0,0.0,0.0,10.0,2.0,14.00,1.0


In [149]:
X_test

,Sexf,Sexm,Ec,Eq,Es,Pid,Pclass,Age,Flia
0,0.0,1.0,0.0,1.0,0.0,892.0,3.0,34.500,0.0
1,1.0,0.0,0.0,0.0,1.0,893.0,3.0,47.000,1.0
2,0.0,1.0,0.0,1.0,0.0,894.0,2.0,62.000,0.0
3,0.0,1.0,0.0,0.0,1.0,895.0,3.0,27.000,0.0
4,1.0,0.0,0.0,0.0,1.0,896.0,3.0,22.000,2.0
5,0.0,1.0,0.0,0.0,1.0,897.0,3.0,14.000,0.0
6,1.0,0.0,0.0,1.0,0.0,898.0,3.0,30.000,0.0
7,0.0,1.0,0.0,0.0,1.0,899.0,2.0,26.000,2.0
8,1.0,0.0,1.0,0.0,0.0,900.0,3.0,18.000,0.0
9,0.0,1.0,0.0,0.0,1.0,901.0,3.0,21.000,2.0


In [146]:
# Luego aplicar el transform al X_test.
Test_transformado = imputador_knn.transform(X_test)
#X_test = pd.DataFrame(Test_transformado, columns = X_test.columns)
X_test=Test_transformado
X_test=pd.DataFrame(X_test)
X_test.columns = ["Sexf","Sexm","Ec","Eq","Es","Pid","Pclass","Age","Flia"]
X_test

,Sexf,Sexm,Ec,Eq,Es,Pid,Pclass,Age,Flia
0,0.0,1.0,0.0,1.0,0.0,892.0,3.0,34.500,0.0
1,1.0,0.0,0.0,0.0,1.0,893.0,3.0,47.000,1.0
2,0.0,1.0,0.0,1.0,0.0,894.0,2.0,62.000,0.0
3,0.0,1.0,0.0,0.0,1.0,895.0,3.0,27.000,0.0
4,1.0,0.0,0.0,0.0,1.0,896.0,3.0,22.000,2.0
5,0.0,1.0,0.0,0.0,1.0,897.0,3.0,14.000,0.0
6,1.0,0.0,0.0,1.0,0.0,898.0,3.0,30.000,0.0
7,0.0,1.0,0.0,0.0,1.0,899.0,2.0,26.000,2.0
8,1.0,0.0,1.0,0.0,0.0,900.0,3.0,18.000,0.0
9,0.0,1.0,0.0,0.0,1.0,901.0,3.0,21.000,2.0


In [148]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sexf    418 non-null    float64
 1   Sexm    418 non-null    float64
 2   Ec      418 non-null    float64
 3   Eq      418 non-null    float64
 4   Es      418 non-null    float64
 5   Pid     418 non-null    float64
 6   Pclass  418 non-null    float64
 7   Age     418 non-null    float64
 8   Flia    418 non-null    float64
dtypes: float64(9)
memory usage: 29.5 KB


#### Búsqueda del mejor modelo

#### 1. Split de X_train y X_test en X_val_train, X_val_test, y_val_train, y_val_test (80,20)


In [54]:
from sklearn.model_selection import train_test_split
X_val_train, X_val_test, y_val_train, y_val_test=train_test_split(X_train, y_train,test_size=0.20,random_state=123)

#### 2. Aplicar diferentes modelos al X_val_train, X_val_test para encontrar el mejor modelo

In [55]:
# Regresión Logística. ##################################################################
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
#modelo_RL = linear_model.LogisticRegression(max_iter= 1000, penalty = 'none', random_state=123)
modelo_RL=linear_model.LogisticRegression(max_iter= 20000,penalty='l2',fit_intercept=True, random_state=123)

In [56]:
hiperp_RL = {"C" : [1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3]}
#hiperp_RL = [1e-4,1e-3,1e-2,1e-1,1,1e1,1e2,1e3]

In [57]:
modelo_RL = GridSearchCV(estimator = modelo_RL, param_grid = hiperp_RL, cv = 5, scoring = "accuracy")

In [58]:
# Entrenamiento con X_val_train e y_val_train.
modelo_RL.fit(X_val_train, y_val_train)

C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

GridSearchCV(cv=5,
             estimator=LogisticRegression(max_iter=20000, random_state=123),
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10.0, 100.0,
                               1000.0]},
             scoring='accuracy')

In [59]:
MR=modelo_RL.best_score_

In [60]:
print("El mejor resultado de Regresión Logística es: ", MR*100)

El mejor resultado de Regresión Logística es:  63.01290259036738


In [61]:
modelo_RL.best_params_

{'C': 100.0}

In [62]:
# Árbol de decisiones.######################################################################
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 20, random_state = 123)

In [63]:
hiperp_tree = {"max_depth" : [1,2,3,4,5,6,7,8,9,10,11,12,13]
              }

In [64]:
modelo_arbol = GridSearchCV(estimator = tree, param_grid = hiperp_tree, cv = 5, scoring = "accuracy")

In [65]:
# Entrenamiento con X_val_train e y_val_train.
modelo_arbol.fit(X_val_train, y_val_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=20, random_state=123),
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13]},
             scoring='accuracy')

In [66]:
MR=modelo_arbol.best_score_

In [67]:
print("El mejor resultado de Modelo Árbol es: ", MR*100)

El mejor resultado de Modelo Árbol es:  62.30670737712991


In [68]:
modelo_arbol.best_params_

{'max_depth': 1}

In [69]:
# KNN.########################################################
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_jobs=-1)

In [70]:
hiperp_knn = {"n_neighbors" : [1,2,3,4,5,6,7,8,9,10,11,12,13],
              "weights" : ['uniform', 'distance'] 
             }

In [71]:
modelo_Knn = GridSearchCV(estimator = knn, param_grid = hiperp_knn, cv = 5, scoring = "accuracy")

In [72]:
# Entrenamiento con X_val_train e y_val_train.
modelo_Knn.fit(X_val_train, y_val_train)

C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y t

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_jobs=-1),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [73]:
MR=modelo_Knn.best_score_

In [74]:
print("El mejor resultado de Knn es: ", MR*100)

El mejor resultado de Knn es:  60.19994090416626


In [75]:
modelo_Knn.best_params_

{'n_neighbors': 2, 'weights': 'uniform'}

In [76]:
#SVM ###################################################################
from sklearn.svm import SVC
svm = SVC( random_state=123)

In [77]:
hiperp_svm = {"C": [0.001, 0.01, 0.1, 1.],
              "kernel": ["linear", "sigmoid"], #["linear", "poly", "rbf", "sigmoid"],
            #  "gamma": ["scale", "auto"], 
             }

In [78]:
modelo_svm = GridSearchCV(estimator = svm, param_grid = hiperp_svm, cv = 5, scoring = "accuracy")

In [79]:
# Entrenamiento con X_val_train e y_val_train.
modelo_svm.fit(X_val_train, y_val_train)

C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

GridSearchCV(cv=5, estimator=SVC(random_state=123),
             param_grid={'C': [0.001, 0.01, 0.1, 1.0],
                         'kernel': ['linear', 'sigmoid']},
             scoring='accuracy')

In [88]:
MR=modelo_svm.best_score_

In [89]:
print("El mejor resultado de SVC es: ", MR*100)

El mejor resultado de SVC es:  62.44755244755245


In [90]:
modelo_svm.best_params_

{'C': 0.1, 'kernel': 'sigmoid'}

In [91]:
# Random Forest.
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 123)

In [93]:
hiperp_rf = {"n_estimators" : [5, 10, 20, 30, 50, 100, 150, 250, 500],
             "max_depth" : [6, 8, 10, 25, 50, 75],
             "criterion" :['gini', 'entropy'],
             "max_features": ['auto', 'sqrt', 'log2'],  
            }

In [94]:
modelo_rf = GridSearchCV(estimator = rf, param_grid = hiperp_rf, cv = 5, scoring = "accuracy")

In [95]:
# Entrenamiento con X_val_train e y_val_train.
modelo_rf.fit(X_val_train, y_val_train)

C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\CTI0212\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:593: DataConversionWarning: A column-vec

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=123),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [6, 8, 10, 25, 50, 75],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 10, 20, 30, 50, 100, 150, 250,
                                          500]},
             scoring='accuracy')

In [96]:
MR=modelo_rf.best_score_

In [97]:
MR=modelo_rf.best_score_

In [98]:
print("El mejor resultado de Random Forest es: ", MR*100)

El mejor resultado de Random Forest es:  64.55727371220328


In [99]:
modelo_rf.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 250}

#### 3. Aplica el mejor modelo encontrado (Random Forest) al X_train y X_test

Random Forest = 64,55 / SVC = 62,44 / Knn = 60,19 / Tree = 62,31 / Regresión Logística = 63,01

In [154]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

In [155]:
y_test.head(4)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0


In [156]:
y_test=y_test.drop(["PassengerId"],axis=1)

In [157]:
Pred_Best=modelo_rf.predict(X_test)

In [158]:
Pred_Best = pd.DataFrame(Pred_Best)
Pred_Best.columns = ["Survived"]

In [159]:
accuracy_Best = accuracy_score(y_test, Pred_Best)
accuracy_Best * 100

78.70813397129187

#### 4. Guardado del modelo.


In [160]:
import pickle
pickle.dump(modelo_rf, open('modelo_Best.sav', 'wb'))

#### 5. Apertura del modelo guardado y prueba con mis datos


In [161]:
# traemos nuestro modelo entrenado
import pickle
with open('modelo_Best.sav', "rb") as file:
    modelo_Best = pickle.load(file)

In [166]:
#import pandas as pd
Nuevo_Pasajero=pd.DataFrame(data=[[0,1,1,0,0,892,2,23,0]], columns=["Sexf","Sexm","Ec","Eq","Es","Pid","Pclass","Age","Flia"])
Nuevo_Pasajero

,Sexf,Sexm,Ec,Eq,Es,Pid,Pclass,Age,Flia
0,0,1,1,0,0,892,2,23,0


In [168]:
Nvo=modelo_Best.predict(Nuevo_Pasajero)

In [176]:
if Nvo[0] == 0:
    print("Me Mori 😢")
else:
    print("Vivi!!!!  🙂")
    

Me Mori 😢
